# __ALS ALGORITHM__

#Name: Pruthvi Raj R G
#Roll NUmber: EE17B114

## To run spark in Colab, we need to first install all the dependencies in Colab environment i.e. Apache Spark 2.3.2 with hadoop 2.7, Java 8 and Findspark to locate the spark in the system. The tools installation can be carried out inside the Jupyter Notebook of the Colab.

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.0-preview2/spark-3.0.0-preview2-bin-hadoop2.7.tgz
!tar -xvf spark-3.0.0-preview2-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-preview2-bin-hadoop2.7"
import findspark
findspark.init()

spark-3.0.0-preview2-bin-hadoop2.7/
spark-3.0.0-preview2-bin-hadoop2.7/data/
spark-3.0.0-preview2-bin-hadoop2.7/data/streaming/
spark-3.0.0-preview2-bin-hadoop2.7/data/streaming/AFINN-111.txt
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/sample_binary_classification_data.txt
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/sample_kmeans_data.txt
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/sample_multiclass_classification_data.txt
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/sample_lda_libsvm_data.txt
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/iris_libsvm.txt
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/pagerank_data.txt
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/sample_linear_regression_data.txt
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/pic_data.txt
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/als/
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/als/test.data
spark-3.0.0-preview2-bin-hadoop2.7/data/mllib/als/sample_movielens_rati

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [4]:
from google.colab import files
files.upload()

Saving ALS.txt to ALS.txt


{'ALS.txt': b'0::2::3\n0::3::1\n0::5::2\n0::9::4\n0::11::1\n0::12::2\n0::15::1\n0::17::1\n0::19::1\n0::21::1\n0::23::1\n0::26::3\n0::27::1\n0::28::1\n0::29::1\n0::30::1\n0::31::1\n0::34::1\n0::37::1\n0::41::2\n0::44::1\n0::45::2\n0::46::1\n0::47::1\n0::48::1\n0::50::1\n0::51::1\n0::54::1\n0::55::1\n0::59::2\n0::61::2\n0::64::1\n0::67::1\n0::68::1\n0::69::1\n0::71::1\n0::72::1\n0::77::2\n0::79::1\n0::83::1\n0::87::1\n0::89::2\n0::91::3\n0::92::4\n0::94::1\n0::95::2\n0::96::1\n0::98::1\n0::99::1\n1::2::2\n1::3::1\n1::4::2\n1::6::1\n1::9::3\n1::12::1\n1::13::1\n1::14::1\n1::16::1\n1::19::1\n1::21::3\n1::27::1\n1::28::3\n1::33::1\n1::36::2\n1::37::1\n1::40::1\n1::41::2\n1::43::1\n1::44::1\n1::47::1\n1::50::1\n1::54::1\n1::56::2\n1::57::1\n1::58::1\n1::60::1\n1::62::4\n1::63::1\n1::67::1\n1::68::4\n1::70::2\n1::72::1\n1::73::1\n1::74::2\n1::76::1\n1::77::3\n1::78::1\n1::81::1\n1::82::1\n1::85::3\n1::86::2\n1::88::2\n1::91::1\n1::92::2\n1::93::1\n1::94::2\n1::96::1\n1::97::1\n2::4::3\n2::6::

## Import necessary packages

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

## "ALS.txt" contains userID, MovieID, Rating seperated by double colons(::)

In [0]:
lines = spark.read.text("./ALS.txt").rdd
parts = lines.map(lambda row: row.value.split("::"))
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]), rating=float(p[2])))
ratings = spark.createDataFrame(ratingsRDD)
(training, test) = ratings.randomSplit([0.8, 0.2])

## Build the recommendation model using ALS on the training data
Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics

In [0]:
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop")
model = als.fit(training)

## Evaluate the model by computing the RMSE on the test data

In [9]:
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.6186433339755346


# RMS error of the fitted model : 1.6186433339755346


## Generate top 10 movie recommendations for each user

In [0]:
userRecs = model.recommendForAllUsers(10)

## Converting to Pandas Dataframe for easy visualization

In [0]:
pandas_userRecs = userRecs.select("*").toPandas()

In [12]:
pandas_userRecs

,userId,recommendations
0,28,"[(74, 5.361251354217529), (81, 5.0317368507385..."
1,26,"[(94, 5.203394889831543), (22, 5.1169037818908..."
2,27,"[(27, 4.334595203399658), (18, 4.0540952682495..."
3,12,"[(20, 5.2602386474609375), (17, 5.107626438140..."
4,22,"[(80, 5.5386762619018555), (68, 5.062793731689..."
5,1,"[(22, 5.104106426239014), (62, 3.7105305194854..."
6,13,"[(93, 3.671485185623169), (4, 3.07506871223449..."
7,6,"[(32, 4.915013790130615), (25, 4.5586280822753..."
8,16,"[(90, 4.858157634735107), (54, 4.7492661476135..."
9,3,"[(30, 5.356832027435303), (69, 5.1149940490722..."


In [0]:
pandas_userRecs.to_csv("ALS out.txt", index = False)

## Experimenting with different train-test splits

## 1. SPLIT - 90/10


In [0]:
(training, test) = ratings.randomSplit([0.9, 0.1])

In [0]:
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop")
model = als.fit(training)

In [16]:
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.28158563494014


## Value of the RMS : 1.28158563494014

## 2. SPLIT - 70/30


In [0]:
(training, test) = ratings.randomSplit([0.7, 0.3])

In [0]:
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop")
model = als.fit(training)

In [19]:
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.9616800817803473


## Value of the RMS : 1.9616800817803473

## 3. SPLIT - 60/40


In [0]:
(training, test) = ratings.randomSplit([0.6, 0.4])

In [0]:
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating", coldStartStrategy="drop")
model = als.fit(training)

In [22]:
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 2.038054388270268


## Value of the RMS : 2.038054388270268